In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")

os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")


In [2]:
from langchain_community.utilities.sql_database import SQLDatabase
db_user = "root"
db_password = "root"
db_host = "localhost"  
db_port = "3306"  
db_name = "classicmodels"
db = SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}")

In [3]:
print(db.dialect)

mysql


In [4]:
print(db.get_usable_table_names())

['customers', 'employees', 'offices', 'orderdetails', 'orders', 'payments', 'productlines', 'products']


In [5]:
print(db.table_info)


CREATE TABLE customers (
	`customerNumber` INTEGER NOT NULL, 
	`customerName` VARCHAR(50) NOT NULL, 
	`contactLastName` VARCHAR(50) NOT NULL, 
	`contactFirstName` VARCHAR(50) NOT NULL, 
	phone VARCHAR(50) NOT NULL, 
	`addressLine1` VARCHAR(50) NOT NULL, 
	`addressLine2` VARCHAR(50), 
	city VARCHAR(50) NOT NULL, 
	state VARCHAR(50), 
	`postalCode` VARCHAR(15), 
	country VARCHAR(50) NOT NULL, 
	`salesRepEmployeeNumber` INTEGER, 
	`creditLimit` DECIMAL(10, 2), 
	PRIMARY KEY (`customerNumber`), 
	CONSTRAINT customers_ibfk_1 FOREIGN KEY(`salesRepEmployeeNumber`) REFERENCES employees (`employeeNumber`)
)DEFAULT CHARSET=utf8mb4 COLLATE utf8mb4_0900_ai_ci ENGINE=InnoDB

/*
3 rows from customers table:
customerNumber	customerName	contactLastName	contactFirstName	phone	addressLine1	addressLine2	city	state	postalCode	country	salesRepEmployeeNumber	creditLimit
103	Atelier graphique	Schmitt	Carine 	40.32.2555	54, rue Royale	None	Nantes	None	44000	France	1370	21000.00
112	Signal Gift Stores	King	Je

In [6]:
from langchain.chains import create_sql_query_chain
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)
generate_query = create_sql_query_chain(llm , db)
query = generate_query.invoke({"question": "what is price of `1968 Ford Mustang`"})
query

"SELECT `buyPrice`, `MSRP`\nFROM products\nWHERE `productName` = '1968 Ford Mustang'\nLIMIT 1;"

In [10]:
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool
execute_query_tool = QuerySQLDataBaseTool(db = db)
execute_query_tool.invoke(query)

"[(Decimal('95.34'), Decimal('194.57'))]"

In [11]:
 from operator import itemgetter

 from langchain_core.output_parsers import StrOutputParser
 from langchain_core.prompts import PromptTemplate
 from langchain_core.runnables import RunnablePassthrough

 answer_prompt = PromptTemplate.from_template(
     """Given the following user question, corresponding SQL query, and SQL result, answer the user question.

 Question: {question}
 SQL Query: {query}
 SQL Result: {result}
 Answer: """
 )

 rephrase_answer = answer_prompt | llm | StrOutputParser()

 chain = (
     RunnablePassthrough.assign(query=generate_query).assign(
         result=itemgetter("query") | execute_query_tool
     )
     | rephrase_answer
 )

 chain.invoke({"question": "what is price of `1968 Ford Mustang"})
#On invoking the chain , question -> generate_query[(llm , db) chain] generates query and stores it in query variable -> itemgetter extracts the query item and passes it to execute_query_tool which executes the query , generates the result and stores it in result variable
#Now there is (question , query , result) inside the chain -> rephrase_answer
#Inside rephrase_answer chain , (question ,query ,answer) -> prompt template -> llm -> output parser -> output

'The price of the 1968 Ford Mustang is $95.34.'

In [12]:
chain.invoke({"question": "How many customers have an order count greater than 5"})

'There are 2 customers who have an order count greater than 5.'

In [14]:
chain.invoke({"question": "Find the top 5 products with the highest total sales revenue"})

'The top 5 products with the highest total sales revenue are:\n1. 1992 Ferrari 360 Spider red - $276,839.98\n2. 2001 Ferrari Enzo - $190,755.86\n3. 1952 Alpine Renault 1300 - $190,017.96\n4. 2003 Harley-Davidson Eagle Drag Bike - $170,686.00\n5. 1968 Ford Mustang - $161,531.48'

In [15]:
chain.invoke({"question": "List the top 3 countries with the highest number of orders"})

'The top 3 countries with the highest number of orders are:\n1. USA with 112 orders\n2. France with 37 orders\n3. Spain with 36 orders'